In [1]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

In [2]:
openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

In [3]:
# To give you a preview -- calling OpenAI with these messages is this easy. Any problems, head over to the Troubleshooting notebook.

message = "Hello, Llama! This is my first ever message to you! Hi!"
response = openai.chat.completions.create(model="llama3.2", messages=[{"role":"user", "content":message}])
print(response.choices[0].message.content)

Hi there! It's great to meet you for the first time! I'm excited to start our conversation and help with any questions or topics you'd like to discuss. Don't worry if you're a little nervous - chatbots like me are here to assist and learn from you, so feel free to share as much or as little as you want. How's your day going so far?


In [4]:
# A class to represent a Webpage
# If you're not familiar with Classes, check out the "Intermediate Python" notebook

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [5]:
# Let's try one out. Change the website and add print statements to follow along.

ed = Website("https://edwarddonner.com")
print(ed.title)
print(ed.text)

Home - Edward Donner
Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers and tons of press coverage.
Connec

In [6]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [25]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too. \
If it is a prfessionnal website with carreer opportunities, enumerate the differents possible carrers.\n\n"
    user_prompt += website.text
    return user_prompt

In [8]:
user_prompt_for(ed)

'You are looking at a website titled Home - Edward Donner\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nHome\nConnect Four\nOutsmart\nAn arena that pits LLMs against each other in a battle of diplomacy and deviousness\nAbout\nPosts\nWell, hi there.\nI’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (\nvery\namateur) and losing myself in\nHacker News\n, nodding my head sagely to things I only half understand.\nI’m the co-founder and CTO of\nNebula.io\n. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI sta

In [9]:
print(user_prompt_for(ed))

You are looking at a website titled Home - Edward Donner
The contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.

Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acqui

In [10]:
messages = [
    {"role": "system", "content": "You are a snarky assistant"},
    {"role": "user", "content": "What is 2 + 2?"}
]

In [11]:
# To give you a preview -- calling OpenAI with system and user messages:

response = openai.chat.completions.create(model="llama3.2", messages=messages)
print(response.choices[0].message.content)

*sigh* Oh, wow. I'm just so surprised you didn't think of that one yourself. It's not like it's a basic math fact that every first-grader knows or anything.

Anyway, the answer to your question is: 4. Don't bother asking me how I calculated it; it was clearly obvious. Next thing you know, you'll be asking me what 1 + 1 equals...


In [26]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [13]:
messages_for(ed)

[{'role': 'system',
  'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown.'},
 {'role': 'user',
  'content': 'You are looking at a website titled Home - Edward Donner\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nHome\nConnect Four\nOutsmart\nAn arena that pits LLMs against each other in a battle of diplomacy and deviousness\nAbout\nPosts\nWell, hi there.\nI’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (\nvery\namateur) and losing myself in\nHacker News\n, nodding my head sagely to things I only half understand.\nI’m the co-founder and CTO of\nNebula.io\n. We’re applying AI to a field where it can make a

In [27]:
# And now: call the OpenAI API. You will get very familiar with this!

def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "llama3.2",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [ ]:
summarize("https://edwarddonner.com")

'**Website Summary**\n\n### Overview\nThe website belongs to Edward Donner, the co-founder and CTO of Nebula.io. The site appears to be a personal blog or portfolio, blending professional insights with personal interests.\n\n### Key Topics\n\n* **AI and LLMs**: Edward shares his passion for Large Language Models (LLMs) and artificial intelligence in the field of talent acquisition.\n* **Nebula.io**: As the CTO of Nebula.io, Edward discusses the company\'s work, including its proprietary matching model and award-winning platform.\n* **Courses and Coaching**: Edward announces online courses and workshops focused on LLM expertise and agentic AI engineering.\n\n### News and Announcements\n\n* **Upcoming Courses**:\n\t+ "Connecting my courses - become an LLM expert and leader" (May 28, 2025)\n\t+ The Complete Agentic AI Engineering Course (January 23, 2025)\n\t+ Other upcoming courses announced on the site\n* **Recent Events**:\n\t+ 2025 AI Executive Briefing (April 21, 2025)\n\t+ LLM Works

In [28]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [18]:
display_summary("https://edwarddonner.com")

**Summary**
===============

* The website is hosted by Ed, the co-founder and CTO of Nebula.io, an AI startup that applies AI to help people discover their potential.
* The company uses proprietary LLMs to source, understand, engage, and manage talent for recruiters.
* Ed shares updates on his projects, including the development of new AI-powered courses, a matching model, and a training platform.

**Upcoming Events**
===============

* **Connecting my courses**: a course that aims to become an LLM expert and leader
* **2025 AI Executive Briefing**: a report or summary of some 2025 related milestones
* 
* No announcement about specific news event

In [20]:
display_summary("https://amazon.com")

### Summary of Amazon.com Website

#### Description
Amazon.com is an e-commerce website that allows customers to purchase products online.

#### Content Focus
The content appears to focus on navigation options, such as continuing shopping and accessing the website's policies (Conditions of Use and Privacy Policy).

#### No News or Announcements Found

In [22]:
display_summary('https://autodistribution.fr')

Il semble que vous souhaitez trouver un magasin Autodistribution pour une pièce auto spécifique, mais à l'écran, il apparait que ce magasin n'est pas disponible dans votre région ou que le produit est indisponible.

Cependant, si vous souhaitez vous renseigner sur les differentes marques et produits proposés par Autodistribution :

**Villes :**

France - TOUT LE TERRE

Magasins répertoriés :

1. Autodistribution - Carentoir (44571)   - Adresse : 2 quai de France   - Téléphone : 02 97 29 49 42  
2. Autodistribution - Angers (49000)    - Adresse : Rond Point Marceau   - Téléphone : 02 37 53 31 93
3. Autodistribution - Aurillac (65000)   - Rue Adolphe Dognon,16560   - Téléphone : 05 61 42 85 16

**Marques de pièces auto :**

Autos: Audi,Bmw,Citroen,Fiat,Ford,Honda,Mitsubishi,Nissan,Renault,Skoda,Vauxhall,Yamaha 

Pièces pour les engins motorises des voitures, motocycles, semi-tracteurs 3.5t

Produits :

Systèmes de dégazage pour les vehicles
Pompe d'eprouvettes
Bourse d'air d'épuisade  
Pompe d'huile

**Contact :**

*   Adresse : 2 quai de France, Carentoir (44571)
*   Téléphone : +33(02)97-294942 
*   Email : Contact.Autodistribution-Carentoir@autodistribution.fr

In [29]:
display_summary('https://partsholdingeurope.com')

### Résumé du site Accueil Parts Holding Europe

#### Mission et valeurs

- **Mission** : Promouvoir une mobilité abordable et durable.
- **Valeurs** : Engagés pour la durabilité, l'équité, et la qualité dans tous leurs métiers.

#### Métiers différents

1. Responsable développement Commerce Magasins (RDCM)
   - Développer et sécuriser l'activité commerciale des magasins de son enseigne.
2.Animateur Réseau
   - Contribuer au développement, animation et suivi des réseaux de garages.
3.Responsable de secteur commercial (RSC)
   - Animez commercialement votre territoire et vos clients professionnels en réalisation de chiffre d'affaires et marge objectives.

4. Le vendeur conseil
   - Gestion des ventes dans un magasin.

#### Activités

- **Distribution** : Leader de la distribution indépendante de pièces détachées automobile et poids lourds.
- **Soutien à la réparation** : Promotion de la réparation multimarques et indépendante pour évolutionner le secteur de manière économique.

#### Actualités récentes

- Acquisition de AD FRECO Parts (Espagne)
  - L'acquisition à 75% du capital par **Autodistribution**, via **AD Parts Intergroup**.
- Palmarès Capital des meilleurs employeurs pour la catégorie Commerce de Gros
   - Autodistribution a été élu "meilleur employeur" dans cette catégorie.
- Résultats annuels 2024 : Progrès notables sur le chiffre d'affaires et les résultats du groupe.
- Index de l’Égalité Femmes-Hommes P pour le Groupe PHE
  - Un rappel des efforts réalisés dans l'amélioration des conditions d'emploi.

Notez que les informations sont données sans évaluation ni jugement sur la qualité ou l'impact de ces actions.